# Машинное обучение: постепенное обучение на больших наборах данных¶

Некоторые оценщики scikit-learn реализуют метод partial_fit. Это означает, что их можно обучать постепенно. Инкрементальная метаоценка в Dask-ML обеспечивает хороший мост между данными, хранящимися в массиве Dask, и оценщиками, реализующими partial_fit.


Подключение к кластеру

In [ ]:
from dask.distributed import Client
client = Client()
client

## Create a random dataset

В этом примере происходит генерация большого случайного массива dask в кластере. На практике данные загружают данные из хранилища данных (таблица SQL, HDFS, облачное хранилище).

In [ ]:
import dask
import dask.array as da
from distributed.utils import format_bytes

import dask_ml.datasets
import dask_ml.model_selection

In [ ]:
n_samples = 4_000_000
n_features = 1_000
chunks = n_samples // 50

X, y = dask_ml.datasets.make_classification(
    n_samples=n_samples, n_features=n_features,
    chunks=chunks,
    random_state=0
)

## Разделение выборки на тестовую и обучающуюся

In [ ]:
X_train, X_test, y_train, y_test = dask_ml.model_selection.train_test_split(
    X, y
)
X_train, X_test, y_train, y_test = dask.persist(
    X_train, X_test, y_train, y_test
)

format_bytes(X_train.nbytes)

## Создание  Scikit-Learn Model, SGDClassifier

Будем использовать SGDClassifier для основного оценщика, реализующего partial_fit.
Incemental будет последовательно передавать ему блоки массива dask.

In [ ]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

estimator = SGDClassifier(
    max_iter=1000,
    random_state=0
)

inc = Incremental(estimator, scoring='accuracy')

In [ ]:
%time _ = inc.fit(X_train, y_train, classes=[0, 1])

## Предсказание модели

Прогнозирование является ленивым и возвращает массив Dask. Обычно предпочтительнее хранить данные в кластере, чем пытаться вернуть большие результаты на одну машину.

In [ ]:
inc.predict(X_test)

Подсчет точности происходит немедленно, но происходит в кластере.

In [ ]:
%time inc.score(X_test, y_test)

Как видно, эта модель не особо хороша, но она смогла очень быстро проанализировать большой объем данных